In [2]:
import sys
import numpy as np
import pymc as pm
import aesara.tensor as at
import tqdm as tqdm
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from scipy import stats
import pandas as pd
import logging
import extract_correct_csv

WARNING (aesara.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [3]:
import warnings
warnings.filterwarnings('ignore')


In [4]:
#scaler = StandardScaler()

#parametri

all_valid_subject = extract_correct_csv.extract_only_valid_subject()

#SUBJECT = input("subject list to extract:")
#subj_ = int(SUBJECT)
num_trials_to_remove = 16#int(input("Number of trials to remove: "))
extraction_method = 'wavelet'#str(input("Extraction method (wavelet/mean)"))
latent_space = 5#int(input("Latent space dimension max: "))
list_latent_space = np.arange(1,latent_space)
use_sampling = 'n'#input("Use sampling? y/n: ")
if use_sampling == 'y':
    use_sampling = True
else:
    use_sampling = False


In [5]:
logging.basicConfig(level=logging.INFO, filename="logfile", filemode="a+",
                        format="%(asctime)-15s %(levelname)-8s %(message)s")
for SUBJECT in all_valid_subject:
    logging.info("\n ---- CHANGE SUBJECT ---- \n")
    logging.info("Subject number: " + str(SUBJECT)+"\n")


    for latent_space in list_latent_space:

        logging.info("\n ---- CHANGE K ----\n")
        logging.info("Subject number: " + str(SUBJECT) + " Latent space dimension: " + str(
            latent_space) + " feat extraction: " + extraction_method + " using sampling: " + str(use_sampling))

        subj_ = extract_correct_csv.read_correct_subject_csv(SUBJECT)
        # extract data rating

        csv_ = '/data/notebook_files/behavior/LookAtMe_0'+str(subj_)+'.csv'
        csv_ = '/home/paolo/matteo/matteo/unimi/tesi_master/code/osfstorage-archive/behavior/LookAtMe_0'+str(subj_)+'.csv'
        global_data = pd.read_csv(csv_, sep='\t')
        y = np.array(list([int(d>2) for d in global_data['rating']]))
        e_labels = y[:,np.newaxis]  # rating > 2

        e_labels = e_labels[num_trials_to_remove:]
        N_e = e_labels.shape[0]
        D_e = e_labels.shape[1]
        #print(N_e,D_e)

        from deepemogp import feature_extractor
        from deepemogp.signal import physio as physio
        from deepemogp import datasets as datasets
        from deepemogp.signal import behavior as behavior
        from deepemogp.signal.physio import ecg
        # from: https://github.com/SheffieldML/GPy
        #import GPy
        # from: https://github.com/SheffieldML/PyDeepGP
        #import deepgp

        import pandas as pd
        import numpy as np

        show = False
        # definition of the feature extractors to be used later
        f1 = feature_extractor.FE('wavelet', window=(2, 1))
        f2 = feature_extractor.FE('mean', window=(1,0))

        # definition of the physiological signals to be extracted
        if extraction_method == 'wavelet':
            eda_ = physio.EDA(f1)
            hr_ = physio.HR(f1)
        else:
            eda_ = physio.EDA(f2)
            hr_ = physio.HR(f2)
        pupil_ = behavior.PUPIL(f2)



        # extraction of the desired data from the dataset
        d = datasets.FEAR(signals={hr_,pupil_,eda_}, subjects={str(SUBJECT)})

        for s in d.signals:
            # preprocess ...
            if s.name =='EDA':
                s.preprocess(show=show,new_fps=500)
                s.feature_ext.extract_feat(s,show=show)
            else:
                if s.name == 'HR':
                    list_hr_test = s.raw[0]['data']
                    s.preprocess(show=show, useneurokit=True)
                    s.feature_ext.extract_feat(s,show=show)

                else:
                    s.feature_ext.extract_feat_without_preprocess(s, show=show)

            #add feature extraction for eda before preprocessing

            # ... and extract features from each signal type


        for sig in d.signals:
            if sig.name=='EDA':
                eda_data = sig.features
            if sig.name=='HR':
                hr_data = sig.features
            if sig.name=='PUPIL':
                pupil_data = sig.features

        TRIAL = 160
        hr =np.array(hr_data)
        hr = hr.reshape((TRIAL, int(hr.shape[0]/TRIAL*hr.shape[1])))
        hr= hr[num_trials_to_remove:]

        pupil = np.array(pupil_data)
        pupil = pupil.reshape((TRIAL, int(pupil.shape[0]/TRIAL*pupil.shape[1])))
        pupil = pupil[num_trials_to_remove:]

        eda = np.array(eda_data)
        eda = eda.reshape((TRIAL,int(eda.shape[0]/TRIAL*eda.shape[1])))
        eda = eda[num_trials_to_remove:]
        '''different types of observable data

        1) gaze
        2) fisio
            2.1) heart rate variability
            2.2) eda phasic value
        3) social anxiety
        4) aspettativa del dolore
        '''
        print(pupil.shape)
        print(hr.shape)
        print(eda.shape)
        N_pupil = pupil.shape[0]
        D_pupil = pupil.shape[1]

        N_hr = hr.shape[0]
        D_hr = hr.shape[1]

        N_eda = eda.shape[0]
        D_eda = eda.shape[1]
        K = latent_space

        print(K)
        print(N_pupil,D_pupil)
        print(N_hr,D_hr)
        print(N_eda,D_eda)
        print(N_e, D_e)

        # model
        with pm.Model() as sPPCA:
            #dati osservabili
            hr_data = pm.MutableData("hr_data", hr.T)
            pupil_data = pm.MutableData("pupil_data", pupil.T)
            eda_data = pm.MutableData("eda_data", eda.T)

            e_data = pm.ConstantData("e_data", e_labels.T)

            #matrici pesi
            Whr = pm.Normal('Whr', mu=at.zeros([D_hr, K]), sigma=2.0 * at.ones([D_hr, K]), shape=[D_hr, K])
            Wpupil = pm.Normal('Wpupil', mu=at.zeros([D_pupil, K]), sigma=2.0 * at.ones([D_pupil, K]), shape=[D_pupil, K])

            Weda = pm.Normal('Weda', mu=at.zeros([D_eda, K]), sigma=2.0 * at.ones([D_eda, K]), shape=[D_eda, K])

            #weight matrix for pain expectation.
            #check mu,sigma,shape
            We = pm.Normal('W_e', mu=at.zeros([D_e, K]), sigma=2.0 * at.ones([D_e,K]), shape=[D_e, K])

            #latent space
            c = pm.Normal('c', mu=at.zeros([N_hr,K]), sigma=at.ones([N_hr,K]), shape=[N_hr,K])

            # dati dell'hrv interpretati come una gaussiana
            x_hr = pm.Normal('x_hr', mu=Whr.dot(c.T), sigma=at.ones([D_hr,N_hr]) ,shape=[D_hr, N_hr], observed=hr_data)
            # dati della dilatazione pupille interpretati come una gaussiana
            x_pupil = pm.Normal('x_pupil', mu=Wpupil.dot(c.T), sigma=at.ones([D_pupil, N_pupil]), shape=[D_pupil, N_pupil], observed=pupil_data)
            #eda
            x_eda = pm.Normal('x_eda',mu=Weda.dot(c.T),sigma= at.ones([D_eda,N_pupil]), shape=[D_eda,N_eda], observed=eda_data)

            # pain expectation. ciò che dovremmo inferire dato c
            # due strade: binary o multiclass (1-4)
            # p = probability of success?
            x_e = pm.Bernoulli('x_e' , p=pm.math.sigmoid(We.dot(c.T)) , shape =[D_e, N_e], observed=e_data)

            #x_hr = pm.Bernoulli('x_hr', p=pm.math.sigmoid(Whr.dot(c.T)), shape=[D_hr, N_hr], observed=hr_data)
            #x_eda = pm.Bernoulli('x_eda', p=pm.math.sigmoid(Weda.dot(c.T)), shape=[D_eda, N_eda], observed=eda_data)

        gv = pm.model_to_graphviz(sPPCA)


        if use_sampling:
            with sPPCA:
                trace = pm.sample(1000,init='advi+adapt_diag',chains=1,progressbar=False,target_accept=0.95)
        else:
            with sPPCA:
                approx = pm.fit(100000, callbacks=[pm.callbacks.CheckParametersConvergence(tolerance=1e-4)])
                trace = approx.sample(500)

        #az.plot_trace(trace);
        with sPPCA:
            # update values of predictors:
            pm.set_data({"pupil_data": pupil,"hr_data":hr,"eda_data":eda})
            # use the updated values and predict outcomes and probabilities:
            posterior_predictive = pm.sample_posterior_predictive(
                trace, random_seed=123)
        e_pred = posterior_predictive.posterior_predictive["x_e"]
        e_pred_mode = np.squeeze(stats.mode(e_pred[0], keepdims=False)[0])[:,np.newaxis]

        train_accuracy_exp = accuracy_score(e_labels, e_pred_mode)

        logging.info("\n Subject number: " + str(SUBJECT) + " Train Accuracy Pain Expectation: " + str(train_accuracy_exp))




 ---- CHANGE SUBJECT ---- 

Subject number: 2


 ---- CHANGE K ----

Subject number: 2 Latent space dimension: 1 feat extraction: wavelet using sampling: False
>> Loading HR for subject 2 and all sessions from dataset FEAR
>> Loading EDA for subject 2 and all sessions from dataset FEAR
>> Loading PUPIL for subject 2 and all sessions from dataset FEAR
>> Processing HR ... using neurokit
>> Extracting wavelet features from HR signal, adopting (2, 1) window ...
>> Processing EDA ...
>> Extracting wavelet features from EDA signal, adopting (2, 1) window ...
>> Extracting mean features from PUPIL signal, adopting (1, 0) window ...
(144, 5)
(144, 60)
(144, 60)
1
144 5
144 60
144 60
144 1


Finished [100%]: Average Loss = 3.6072e+09
Sampling: [x_e, x_eda, x_hr, x_pupil]



 Subject number: 2 Train Accuracy Pain Expectation: 0.5138888888888888

 ---- CHANGE K ----

Subject number: 2 Latent space dimension: 2 feat extraction: wavelet using sampling: False
>> Loading HR for subject 2 and all sessions from dataset FEAR
>> Loading PUPIL for subject 2 and all sessions from dataset FEAR
>> Loading EDA for subject 2 and all sessions from dataset FEAR
>> Processing HR ... using neurokit
>> Extracting wavelet features from HR signal, adopting (2, 1) window ...
>> Extracting mean features from PUPIL signal, adopting (1, 0) window ...
>> Processing EDA ...
>> Extracting wavelet features from EDA signal, adopting (2, 1) window ...
(144, 5)
(144, 60)
(144, 60)
2
144 5
144 60
144 60
144 1


Finished [100%]: Average Loss = 5.3338e+07
Sampling: [x_e, x_eda, x_hr, x_pupil]



 Subject number: 2 Train Accuracy Pain Expectation: 0.5486111111111112

 ---- CHANGE K ----

Subject number: 2 Latent space dimension: 3 feat extraction: wavelet using sampling: False
>> Loading HR for subject 2 and all sessions from dataset FEAR
>> Loading PUPIL for subject 2 and all sessions from dataset FEAR
>> Loading EDA for subject 2 and all sessions from dataset FEAR
>> Processing HR ... using neurokit
>> Extracting wavelet features from HR signal, adopting (2, 1) window ...
>> Extracting mean features from PUPIL signal, adopting (1, 0) window ...
>> Processing EDA ...
>> Extracting wavelet features from EDA signal, adopting (2, 1) window ...
(144, 5)
(144, 60)
(144, 60)
3
144 5
144 60
144 60
144 1


...

Finished [100%]: Average Loss = 1.6422e+07
Sampling: [x_e, x_eda, x_hr, x_pupil]



 Subject number: 4 Train Accuracy Pain Expectation: 0.5486111111111112

 ---- CHANGE K ----

Subject number: 4 Latent space dimension: 4 feat extraction: wavelet using sampling: False
>> Loading PUPIL for subject 4 and all sessions from dataset FEAR
>> Loading EDA for subject 4 and all sessions from dataset FEAR
>> Loading HR for subject 4 and all sessions from dataset FEAR
>> Extracting mean features from PUPIL signal, adopting (1, 0) window ...
>> Processing EDA ...
>> Extracting wavelet features from EDA signal, adopting (2, 1) window ...
>> Processing HR ... using neurokit
>> Extracting wavelet features from HR signal, adopting (2, 1) window ...
(144, 5)
(144, 60)
(144, 60)
4
144 5
144 60
144 60
144 1


Finished [100%]: Average Loss = 1.3841e+07
Sampling: [x_e, x_eda, x_hr, x_pupil]



 Subject number: 4 Train Accuracy Pain Expectation: 0.5486111111111112

 ---- CHANGE SUBJECT ---- 

Subject number: 10


 ---- CHANGE K ----

Subject number: 10 Latent space dimension: 1 feat extraction: wavelet using sampling: False


TypeError: can only concatenate str (not "int") to str